## How to generate neat problems of Gram-Schmidt Process for students to work on?

Let $U = \{u_1,u_2,...,u_k\}$, where $u_i\in \mathbf{R}^n$. Let $V = \{v_1,v_2,...,v_k\}$ be the orthogonal basis generated by the standard Gram-Schmidt process. According to the process, we have the following formulae:

$\quad v_1=u_1$

$\quad v_2=u_2 - \frac{u_2^T v_1}{\lVert v_1 \lVert ^2}v_1$

$\quad ...$

$\quad v_i=u_i - \frac{u_i^T v_1}{\lVert v_1 \lVert ^2}v_1 - \frac{u_i^T v_2}{\lVert v_2 \lVert ^2}v_2 - ... - \frac{u_i^T v_{i-1}}{\lVert v_{i-1} \lVert ^2}v_{i-1}$

$\quad ...$

For the neatness of the problems generated, the following constraints are imposed:

$\quad \quad \frac{u_i^T v_j}{\lVert v_j \lVert ^2} \in \mathbf{N}-\{0\} \space \forall i>j$

$\Leftrightarrow \quad v_j^T u_i = c\lVert v_j \lVert ^2 \space \exists c \in \mathbf{N}-\{0\} \space \forall i>j$

$\Leftrightarrow \quad \begin{bmatrix} v_1^T \\ v_2^T \\ \vdots \\ v_{i-1}^T \end{bmatrix} u_i = \begin{bmatrix} c_1 \lVert v_1 \lVert ^2 \\ c_2 \lVert v_2 \lVert ^2 \\ \vdots \\ c_{i-1} \lVert v_{i-1} \lVert ^2 \end{bmatrix} \space \exists c_j \in \mathbf{N}-\{0\} \space \forall j=1,2,...,i-1 \space \forall i=2,3,...,k \quad$  ---- (1)

An algorithm is thus designed to generate $U$: first generate $u_1=v_1$, then for $i=2,3,...,k$, generate $u_i$ according to system (1) and then $v_i$ according to the formulae of the process.

For any $i=2,3,...,k$, assume that $c_j$ are given for any $j=1,2,...,i-1$. Since $V$ is defined to be a linearly independent set, $rank\left(\begin{bmatrix} v_1^T \\ v_2^T \\ \vdots \\ v_{i-1}^T \end{bmatrix}\right)=i-1$, i.e. the matrix is of full row rank. Hence, system (1) is consistent and $nullity\left(\begin{bmatrix} v_1^T \\ v_2^T \\ \vdots \\ v_{i-1}^T \end{bmatrix}\right)=n-i+1$, meaning that the number of free variables is $n-i+1$.

To obtain a particular solution of system (1), the following assignment is performed: $u_j = s_j \space \exists s_j \in \mathbf{N} \space \forall j=i,i+1,...,n$ , i.e. to set $u_i, u_{i+1}, ..., u_n$ as the free variables of the equation. Let $s = \begin{bmatrix} s_i \\ s_{i+1} \\ \vdots \\ s_n \end{bmatrix}$. Then, system (1) becomes:

$\quad \begin{bmatrix} v_1 [1:i-1]^T & v_1 [i:n]^T \\ v_2 [1:i-1]^T & v_2 [i:n]^T \\ \vdots \\ v_{i-1} [1:i-1]^T & v_{i-1} [i:n]^T \end{bmatrix} \begin{bmatrix} u_i [1:i-1] \\ s \end{bmatrix} = \begin{bmatrix} c_1 \lVert v_1 \lVert ^2 \\ c_2 \lVert v_2 \lVert ^2 \\ \vdots \\ c_{i-1} \lVert v_{i-1} \lVert ^2 \end{bmatrix} \space \exists c_j \in \mathbf{N}-\{0\} \space \forall j=1,2,...,i-1 \space \forall i=2,3,...,k \quad$, or

$\quad \begin{bmatrix} v_1 [1:i-1]^T \\ v_2 [1:i-1]^T \\ \vdots \\ v_{i-1} [1:i-1]^T \end{bmatrix} u_i [1:i-1] = \begin{bmatrix} c_1  \lVert v_1 \lVert ^2 -v_1 [i:n]^T s \\ c_2 \lVert v_2 \lVert ^2 -v_2 [i:n]^T s \\ \vdots \\ c_{i-1} \lVert v_{i-1} \lVert ^2 -v_{i-1} [i:n]^T s \end{bmatrix} \space \exists c_j \in \mathbf{N}-\{0\} \space \forall j=1,2,...,i-1 \space \forall i=2,3,...,k \quad$ ---- (2)

Note that a random assignment of $s_i, s_{i+1}, ..., s_n$ can result in no solution or infinitely many solutions to the original equation. If this happens, then for simplicity the whole problem is discarded. A new problem will be generated.

In [1]:
# import necessary packages
import numpy as np
import sympy as sp
from sympy.solvers import solve
from sympy import BlockMatrix, GramSchmidt
from sympy.printing.str import StrPrinter
import random
from numpy.random import randint

In [2]:
# Function that prints sympy matrices
def printM(M):
    printer = StrPrinter()
    print(M.table(printer, align='center'))

In [3]:
# Main function
def GramSchmidtGen(n, k):
    
    # Generate v1 = u1
    u = sp.Matrix(randint(-2,2,n))
    while (u[0]==0 or all(u[i]==0 for i in range(1,n))):
        u = sp.Matrix(randint(-2,2,n))
    
    V = [u]  # will also store v2, ..., vk
    U = [u]  # will also store u2, ..., uk
    block = u  # will be the block matrix [u1, ..., ui]
    
    for i in range(2, k+1):
        
        u = sp.zeros(n,1)  # initialize ui
        
        while (sp.Matrix(sp.BlockMatrix([[block, u]])).rank() < i):  # test linear independence
                                                                     # of the set U \union {ui}
            
            c = randint(-2,2,i-1)  # the randomly generated c1, c2, ..., c{i-1} in the proof above
            while (not all(k!=0 for k in c)):  # test if all c's are nonzero
                c = randint(-2,2,i-1)
            
            # construct the RHS vector of the equation (2) in the proof
            b = []
            for j in range(0,i-1):
                b.append(c[j]*(V[j].norm()**2))
            b = sp.Matrix(b)
            s = sp.Matrix(randint(-1,1,n-i+1))
            b = b - block.T[:,i-1:] * s
            
            # solve equation (2) for ui
            u = sp.linsolve((block.T[:,0:i-1], b), sp.symbols('u0:n'))
            
            # discard if equation (2) has no solution or infinitely many solution
            if (type(u)==sp.EmptySet or not (all (str(u)[i]!='u' for i in range(len(str(u)))))):
                return GramSchmidtGen(n, k)

            # construct ui by concatenating with s
            u = sp.Matrix(list(u)).T
            u = sp.Matrix(sp.BlockMatrix([[u],[s]]))
            
            # turn ui into a integral vector
            u = 1/sp.gcd(tuple(u))*u
 
            # discard the generated problem if not all entries of the block matrix are
            # strictly less than 10 in magnitude to make the problems generated neat
            if (not all(abs(u[x]) < 10 for x in range(n))):
                return GramSchmidtGen(n, k)

        U.append(u)  # add ui into U
        block = sp.Matrix(sp.BlockMatrix([[block, u]]))  # add ui into the block matrix
        V.append(sp.GramSchmidt(U)[i-1])  # add vi into V
    
    for i in range(n):
        if (all(block[i,j]==0 for j in range(k))):
            return GramSchmidtGen(n,k)
    
    return U

In [4]:
# For program testing
import time
tStart = time.time()

for i in range(5):  # parameters to be determined by user: number of problems to be generated
    X = GramSchmidtGen(5,4)  # parameters to be determined by user: n and k
    Y = sp.GramSchmidt(X, True)
    
    print("Problem")
    printM(sp.BlockMatrix([[X[i] for i in range(len(X))]]).as_explicit())
    print("Solution")
    printM(sp.BlockMatrix([[Y[i] for i in range(len(Y))]]).as_explicit())
    print("")

tEnd = time.time()
print("Running time:", tEnd - tStart, "s")

Problem
[-1, 3 , -6, -1]
[-1, -1, 2 , 3 ]
[0 , 0 , -1, 2 ]
[1 , -1, -1, 0 ]
[0 , -1, -1, 0 ]
Solution
[-sqrt(3)/3, 2/3 ,  -sqrt(15)/9  ,   -sqrt(11010)/6606  ]
[-sqrt(3)/3, -2/3,  -sqrt(15)/45 ,  71*sqrt(11010)/33030]
[    0     ,  0  ,  -sqrt(15)/15 ,  43*sqrt(11010)/5505 ]
[sqrt(3)/3 ,  0  , -2*sqrt(15)/15,  11*sqrt(11010)/5505 ]
[    0     , -1/3, -8*sqrt(15)/45, -76*sqrt(11010)/16515]

Problem
[1, -2, -2, 3 ]
[0, -1, 1 , 1 ]
[1, 0 , -1, 1 ]
[1, -1, 0 , -1]
[0, 0 , 0 , -1]
Solution
[sqrt(3)/3, -sqrt(3)/3, -sqrt(3)/3,  0  ]
[    0    , -sqrt(3)/3, sqrt(3)/3 , 1/2 ]
[sqrt(3)/3, sqrt(3)/3 ,     0     , 1/2 ]
[sqrt(3)/3,     0     , sqrt(3)/3 , -1/2]
[    0    ,     0     ,     0     , -1/2]

Problem
[1 , 3 , -4, -5]
[-1, -1, 4 , -3]
[1 , -1, 0 , 5 ]
[0 , 0 , -1, -1]
[-1, -1, 0 , -1]
Solution
[1/2 , sqrt(2)/2 ,  0  ,  -sqrt(6)/6 ]
[-1/2,     0     , 2/3 , -2*sqrt(6)/9]
[1/2 , -sqrt(2)/2,  0  ,  -sqrt(6)/6 ]
[ 0  ,     0     , -1/3, -2*sqrt(6)/9]
[-1/2,     0     , -2/3,  -sqrt(6)/9 ]

P